# 2.5 (Zusatz) 2-dim Array -- Image bearbeiten

In der Erweiterung des vorherigen Beispiels wird hier ein Bild in den Speicher geladen wird. Als Nutzer und Programmierer abstrahieren wir ein Bild als zwei-dimensionales Feld von Pixeln (bzw. mehr-dimensional wenn wir vers. Farbwerte betrachten). Im Speicher wird dies aber als ein großer 1-dimensionaler Array angelegt.

Im folgenden werden dazu erst mehrere Funktionen als Hilfe eingeführt:

* zum speichern eines Bildes als PNG
* zum öffnen eines Bildes aus einem BMP Format
* zum Anzeigen

In [ ]:
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "include/stb_image_write.h"
#include <iostream>
#include <fstream>

// Function to write a two-dimensional grayscale image to PNG file
bool writeGrayscaleImage(const char* filename, unsigned char** image, int width, int height)
{
    // Open file for writing
    FILE* file = fopen(filename, "wb");
    if (!file)
    {
        return false;
    }

    // Create a new buffer to store the image data in PNG format
    unsigned char* png_data = new unsigned char[width * height];

    // Copy and flip the image data to match PNG format
    for (int y = 0; y < height; y++)
    {
        for (int x = 0; x < width; x++)
        {
            png_data[(height - 1 - y) * width + x + 100] = image[y][x];
        }
    }

    // Write grayscale image to PNG file
    int result = stbi_write_png(filename, width, height, 1, png_data, 0);

    // Clean up memory
    delete[] png_data;
    fclose(file);

    return result != 0;
}

In [ ]:
void loadBMPImage(const char* file_name, unsigned char**& image, int& width, int& height)
{
    // Open the BMP image file in binary mode
    std::ifstream file(file_name, std::ios::binary);

    if (!file)
    {
        std::cout << "Failed to open " << file_name << std::endl;
        return;
    }

    // Read the header of the BMP image file (54 bytes)
    char header[54];
    file.read(header, 54);

    // Extract the width and height of the image from the header
    width = *(int*)&header[18];
    height = *(int*)&header[22];

    // Calculate the number of padding bytes per row
    int padding = 0;
    while ((width + padding) % 4 != 0)
    {
        padding++;
    }

    // Calculate the total size of the image data in bytes
    int data_size = (width + padding) * height;

    // Allocate memory to store the image data
    unsigned char* data = new unsigned char[data_size];

    // Read the image data from the file into the data buffer
    file.read(reinterpret_cast<char*>(data), data_size);

    // Close the BMP image file
    file.close();

    // Create a 2D array to store the image data
    image = new unsigned char*[height];
    for (int y = 0; y < height; ++y)
    {
        image[y] = &data[(width + padding) * y];
    }
}

<div class="alert alert-warning">

<b>ToDo</b>: 

* Implementieren sie eine Funktion, die den mittleren Graustufenwert über dem Bild berechnet. 
* Im Beispiel ist dies Bild sehr dunkel. Wie können sie dies aufhellen? Erweitern sie den Code entsprechend: Passen sie den Code so an, dass sie die Werte des Bildes im Array sinnvoll aufhellen (den Wertebereich auf das ganze Spektrum für Grauwerte von 0 bis 255 ausdehnen).

</div>

In [ ]:
// Function to calculate the mean grayscale value of an image
double calculateMeanGrayscaleValue2d(unsigned char** image, int width, int height)
{
    double sum = 0.0;
    for (int y = 0; y < height; ++y)
    {
        for (int x = 0; x < width; ++x)
        {
            sum += image[y][x];
        }
    }
    return sum / (width * height);
}

In [ ]:
int main()
{
    const char* file_name = "marie_darkened.bmp";
    unsigned char** image = nullptr;
    int width = 0;
    int height = 0;

    loadBMPImage(file_name, image, width, height);

    // Calculate the mean grayscale value of the image
    double mean = calculateMeanGrayscaleValue2d(image, width, height);
    std::cout << "Mean grayscale value: " << mean << std::endl;

    // Write the grayscale image to PNG file
    const char* filename = "output.png";
    writeGrayscaleImage(filename, image, width, height);

    // Clean up memory
    delete[] image[0];
    delete[] image;
    return 0;
}

## Anzeige des erzeugten Outputs 

Im folgenden weiterer Code zur Anzeige des erzeugten PNG Bildes.

In [ ]:
#include <string>
#include <fstream>

#include "nlohmann/json.hpp"

#include "xtl/xbase64.hpp"

namespace nl = nlohmann;

namespace im
{
    struct image
    {   
        inline image(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);   
            m_buffer << fin.rdbuf();
        }
        
        std::stringstream m_buffer;
    };
    
    nl::json mime_bundle_repr(const image& i)
    {
        auto bundle = nl::json::object();
        bundle["image/png"] = xtl::base64encode(i.m_buffer.str());
        return bundle;
    }
}

In [ ]:
im::image example_im("images/output.png");
example_im